In [4]:
from os import path
import glob
import ffmpeg
import re

In [5]:
folder = 'testVideos/constructive_disagreement/r20230929_1040_student_012'
folder = 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80'
files = glob.glob(folder+"/*")
files.sort()
files

['testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-4e7cc332-2c0b-4bb6-b98c-91d8a7386db9-cam-audio-1679598529333.webm',
 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-4e7cc332-2c0b-4bb6-b98c-91d8a7386db9-cam-video-1679598529336',
 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-a05002dc-95f4-4694-be11-d671741fbc96-cam-audio-1679598528588.webm',
 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-a05002dc-95f4-4694-be11-d671741fbc96-cam-video-1679598528591',
 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-c7c8389a-0587-492e-bebf-051805057a40-cam-audio-1679598528593.webm',
 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-c7c8389a-0587-492e-bebf-051805057a40-cam-video-1679598528595']

In [6]:

group_data = {}
for filepath in files:
    basename = path.basename(filepath)
    participant_id = "-".join(basename.split("-")[1:-3])
    track_type = basename.split("-")[-2]
    track_start = int(basename.split("-")[-1].split(".")[0])
    group_start = int(basename.split("-")[0])
    
    if participant_id not in group_data:
        group_data[participant_id] = {
        "participant_id": participant_id,
        "group_start": int(basename.split("-")[0]),
        "dirname": path.dirname(filepath),
    }

    if track_type == "video":    
        probe = ! ffprobe $filepath 
        line = [line  for line in probe if "Stream " in line][0]
        resolution = re.search(r'\d+x\d+', line).group(0)
        group_data[participant_id][track_type+"_resolution"] = resolution


    group_data[participant_id][track_type] = filepath
    group_data[participant_id][track_type+"_start"] = track_start
    group_data[participant_id][track_type+"_offset"] = track_start - group_start


group_data


{'4e7cc332-2c0b-4bb6-b98c-91d8a7386db9': {'participant_id': '4e7cc332-2c0b-4bb6-b98c-91d8a7386db9',
  'group_start': 1679598527604,
  'dirname': 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80',
  'audio': 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-4e7cc332-2c0b-4bb6-b98c-91d8a7386db9-cam-audio-1679598529333.webm',
  'audio_start': 1679598529333,
  'audio_offset': 1729,
  'video_resolution': '1280x720',
  'video': 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-4e7cc332-2c0b-4bb6-b98c-91d8a7386db9-cam-video-1679598529336',
  'video_start': 1679598529336,
  'video_offset': 1732},
 'a05002dc-95f4-4694-be11-d671741fbc96': {'participant_id': 'a05002dc-95f4-4694-be11-d671741fbc96',
  'group_start': 1679598527604,
  'dirname': 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80',
  'audio': 'testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-a05002dc-95f4-4694-be11-d671741fbc96-cam-audio-1679598528588.webm',
  'audio_start': 167

In [3]:
for participant_id in group_data:
    audio = group_data[participant_id]["audio"]
    video = group_data[participant_id]["video"]
    output = group_data[participant_id]["dirname"] + "/" + participant_id + "_combined.webm"

    ! ffmpeg -i $video -i $audio -map 0:v -map 1:a -c copy -copyts $output

https://superuser.com/a/277667
```
ffmpeg -i video.mp4 -i audio.wav -c:v copy -c:a aac output.mp4
```
Flags
`-i` 

ffmpeg -i video.ext -i audio.ext -map 0:v -map 0:a output.mp4
-i video.ext specified first input
-i audio.ext specified second input
-map 0:v maps video streams in first input (index 0) to output.mp4
-map 1:a maps audio streams in second input (index 1) to output.mp4
output.mp4 is the output file name


Mosaic videos:
https://stackoverflow.com/a/33764934/6361632
https://trac.ffmpeg.org/wiki/FilteringGuide#Multipleinputoverlayin2x2grid


```
ffmpeg -i input0 -i input1 -i input2 -i input3 -filter_complex "[0:v][1:v][2:v][3:v]xstack=inputs=4:layout=0_0|w0_0|0_h0|w0_h0[v]" -map "[v]" output
```

In [64]:
audio0 = group_data["887f298a-d006-4ac1-8636-a41a7d6c7475"]['audio']
video0 = group_data["887f298a-d006-4ac1-8636-a41a7d6c7475"]['video']


In [66]:
! ffmpeg -i $video0 -i $audio0 -map 0:v -map 1:a  test4.mov

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [69]:
! ffmpeg -i $video0 -i $audio0 -map 0:v -map 1:a -c copy test5.webm

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [76]:
! ffmpeg -i $video0 -i $audio0 -map 0:v -map 1:a -c copy -copyts test6.webm

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

To split a stereo to mono:
```
ffmpeg -i stereo.wav -filter_complex "[0:a]channelsplit=channel_layout=stereo:channels=FR[right]" -map "[right]" front_right.wav
```

To merge 6 mono tracks to one multichannel output
```
ffmpeg -i front_left.wav -i front_right.wav -i front_center.wav -i lfe.wav -i back_left.wav -i back_right.wav \
-filter_complex "[0:a][1:a][2:a][3:a][4:a][5:a]join=inputs=6:channel_layout=5.1[a]" -map "[a]" output.wav
```

In [74]:
! ffprobe $audio0

ffprobe version 6.0 Copyright (c) 2007-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --d

In [75]:
! ffprobe $video0

ffprobe version 6.0 Copyright (c) 2007-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --d

In [84]:
audio_inputs = [f'-i {group_data[i]["audio"]}' for i in group_data]
' '.join(audio_inputs)
inputs = ' '.join(audio_inputs)

# selectors = [f'[{i}:a]' for i in range(len(audio_inputs))]
# filter = ''.join(selectors)+f' amerge=inputs={len(audio_inputs)}'
# filter


maps = " ".join([f'-map {i}:a' for i in range(len(audio_inputs))])
maps

'-map 0:a -map 1:a -map 2:a -map 3:a -map 4:a -map 5:a'

In [85]:
! ffmpeg $inputs $maps -c copy -copyts test7.webm

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [ ]:
ffmpeg $inputs -filter_complex "[0:a][1:a][2:a][3:a][4:a][5:a]join=inputs=6:channel_layout=5.1[a]" -map "[a]" output.wav

In [88]:
! ffprobe test6.webm

ffprobe version 6.0 Copyright (c) 2007-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --d

In [89]:
! ffprobe test7.webm

ffprobe version 6.0 Copyright (c) 2007-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --d

In [96]:
inputs = ' '.join([f'-i {input}' for input in files])
inputs

'-i testVideos/constructive_disagreement/r20230929_1040_student_002/1695990112785-04f87da1-2360-4502-974f-471845f6b403-cam-audio-1695991287722 -i testVideos/constructive_disagreement/r20230929_1040_student_002/1695990112785-04f87da1-2360-4502-974f-471845f6b403-cam-video-1695991287726 -i testVideos/constructive_disagreement/r20230929_1040_student_002/1695990112785-0f2a2497-3b29-45a4-976b-00439e04ae2d-cam-audio-1695990115644 -i testVideos/constructive_disagreement/r20230929_1040_student_002/1695990112785-0f2a2497-3b29-45a4-976b-00439e04ae2d-cam-video-1695990116222 -i testVideos/constructive_disagreement/r20230929_1040_student_002/1695990112785-35ca4b46-008e-4f7d-86bb-f0cf18ba5e0e-cam-audio-1695990185122 -i testVideos/constructive_disagreement/r20230929_1040_student_002/1695990112785-35ca4b46-008e-4f7d-86bb-f0cf18ba5e0e-cam-video-1695990185126 -i testVideos/constructive_disagreement/r20230929_1040_student_002/1695990112785-64cecbcd-1c55-45d9-8982-f4c040de001c-cam-audio-1695990113682 -i te

In [97]:
maps = " ".join([f'-map {i}:{"v" if "video" in input else "a"}' for i, input in enumerate(files)])
maps


'-map 0:a -map 1:v -map 2:a -map 3:v -map 4:a -map 5:v -map 6:a -map 7:v -map 8:a -map 9:v -map 10:a -map 11:v'

In [98]:
! ffmpeg $inputs $maps -c copy -copyts test8.webm

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [100]:
! ffprobe test8.webm

ffprobe version 6.0 Copyright (c) 2007-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --d

In [101]:
! ffplay test8.webm

ffplay version 6.0 Copyright (c) 2003-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [38]:
# https://ffmpeg.org/ffmpeg-filters.html#xstack-1


v1 = group_data[list(group_data.keys())[1]]['video']
v2 = group_data[list(group_data.keys())[4]]['video']

! ffmpeg -i $v1 -i $v2 -filter_complex hstack=inputs=2 -vsync 0 -enc_time_base -1 test11.webm

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [46]:
# https://ffmpeg.org/ffmpeg-filters.html#xstack-1


# v1 = group_data[list(group_data.keys())[1]]['video']
v1 = "testVideos/constructive_disagreement/r20230929_1040_student_012/56cb9c8e-f466-41d0-82a6-1955bea4dfaa_combined.webm"
v2 = "testVideos/constructive_disagreement/r20230929_1040_student_012/ac94f535-ef7b-4786-be39-f1df1a5b6995_combined.webm"

! ffmpeg -i $v1 -i $v2 -filter_complex "[0:v]scale=1280:720[v0];[1:v]scale=1280:720[v1];[v0][v1]hstack=inputs=2" -vsync 0 -enc_time_base -1 test13.webm

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [48]:
# https://ffmpeg.org/ffmpeg-filters.html#xstack-1


# v1 = group_data[list(group_data.keys())[1]]['video']
v1 = "testVideos/constructive_disagreement/r20230929_1040_student_012/56cb9c8e-f466-41d0-82a6-1955bea4dfaa_combined.webm"
v2 = "testVideos/constructive_disagreement/r20230929_1040_student_012/ac94f535-ef7b-4786-be39-f1df1a5b6995_combined.webm"

! ffmpeg -i $v1 -i $v2 -filter_complex "[0:v]scale=1280:720[v0];[1:v]scale=1280:720[v1];[v0][v1]hstack=inputs=2[v];[0:a][1:a]amerge=inputs=2,pan=mono|c0<c0+c1+c2+c3|[a]" -map "[a]" -map "[v]" test14.webm

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

ffmpeg filters are made of chains of different filters. Different chains are sesparated by a semicolon, filters within a chain by a comma
filter consists of selectors (e.g.`[0:a][1:a]` etc.) then filter commands, then outputs []


Want the filter to:
## combine all audio tracks into one audio track
- use the amerge filter to take audio streams from different files and add them as separate streams to the output
```
[0:a][1:a]amerge=inputs=2,
```
the output of this can be fed to a single stereo stream:
```
pan=mono|c0<c0+c1+c2+c3|
```

## - xstack videos

- align all audio and video streams


In [ ]:
a1 = "count15.m4a"
a2 = "sayLetters.m4a"

! ffmpeg -i $a1 -i $a2 -filter_complex "[0:a][1:a]amerge=inputs=2,pan=mono|c0<c0+c1|[a]" -map "[a]" test15.m4a

In [7]:
# https://ffmpeg.org/ffmpeg-filters.html#xstack-1

# join_players = ['56cb9c8e-f466-41d0-82a6-1955bea4dfaa', 'ac94f535-ef7b-4786-be39-f1df1a5b6995']
join_players = group_data.keys()

audio_starts = [group_data[i]['audio_start'] for i in join_players]
earliest_audio = audio_starts.index(min(audio_starts))



# format inputs
audio_inputs = [f'-isync {earliest_audio} -i {group_data[i]["audio"]}' for i in join_players]

# format audio chain
audioChain = "".join([f'[{i}:a]' for i in range(len(audio_inputs))]) + \
    f'amerge=inputs={len(audio_inputs)},' + \
    f'pan=mono|c0<{ "+".join([f"c{j}" for j in range(len(audio_inputs)*2)])}|[a]'




# cmd = f'ffmpeg {audio_inputs} {video_inputs} -filter_complex "{audioChain};{videoChain}" -map "[a]" -map "[v]" test15.webm'
cmd = f'ffmpeg {" ".join(audio_inputs)} -filter_complex "{audioChain}" -map "[a]" test16.webm'
print(cmd)

! $cmd -y
# ! ffmpeg $inputs -filter_complex $filter -map "[a]" -map "[v]" test15.webm

# selectors = [f'[{i}:v]scale' for i in range(len(video_inputs))]
# # filter = ''.join(selectors)+f' amerge=inputs={len(audio_inputs)}'
# # filter


# maps = " ".join([f'-map {i}:a' for i in range(len(audio_inputs))])
# maps

# ! ffmpeg -i $v1 -i $v2 -filter_complex hstack=inputs=2 test9.webm

ffmpeg -isync 1 -i testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-4e7cc332-2c0b-4bb6-b98c-91d8a7386db9-cam-audio-1679598529333.webm -isync 1 -i testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-a05002dc-95f4-4694-be11-d671741fbc96-cam-audio-1679598528588.webm -isync 1 -i testVideos/bad_influence/01GW7XY5S7ZZBHB3TW6HCJ9K80/1679598527604-c7c8389a-0587-492e-bebf-051805057a40-cam-audio-1679598528593.webm -filter_complex "[0:a][1:a][2:a]amerge=inputs=3,pan=mono|c0<c0+c1+c2+c3+c4+c5|[a]" -map "[a]" test16.webm
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --

In [76]:
# https://ffmpeg.org/ffmpeg-filters.html#xstack-1

# compute which video index is the earliest, and use it to sync everything else
video_starts = [group_data[i]['video_start'] for i in group_data]
earliest_video = video_starts.index(min(video_starts))

audio_starts = [group_data[i]['audio_start'] for i in group_data]
earliest_audio = audio_starts.index(min(audio_starts))

videoFirst = video_starts[earliest_video] <= audio_starts[earliest_audio]
syncIndex = earliest_video + len(audio_starts) if videoFirst else earliest_audio

# format inputs
audio_inputs = [f'-isync {syncIndex} -i {group_data[i]["audio"]}' for i in group_data]
video_inputs = [f'-isync {syncIndex} -i {group_data[i]["video"]}' for i in group_data]
# inputs = ' '.join(audio_inputs) + ' ' + ' '.join(video_inputs)

# format audio chain
audioChain = "".join([f'[{i}:a]' for i in range(len(audio_inputs))]) + \
    f'amerge=inputs={len(audio_inputs)},' + \
    f'pan=mono|c0<{ "+".join([f"c{j}" for j in range(len(audio_inputs)*2)])}|[a]'

# format video chain
videoChain = "".join([f'[{i+len(audio_inputs)}:v]scale=1280:720[v{i}];' for i in range(len(video_inputs))]) + \
    f'{"".join([f"[v{i}]" for i in range(len(video_inputs))])}hstack=inputs={len(video_inputs)}[v]'



# cmd = f'ffmpeg {audio_inputs} {video_inputs} -filter_complex "{audioChain};{videoChain}" -map "[a]" -map "[v]" test15.webm'
cmd = f'ffmpeg {" ".join(audio_inputs)} -filter_complex "{audioChain}" -map "[a]" test16.webm'
cmd
# ! ffmpeg $inputs -filter_complex $filter -map "[a]" -map "[v]" test15.webm

# selectors = [f'[{i}:v]scale' for i in range(len(video_inputs))]
# # filter = ''.join(selectors)+f' amerge=inputs={len(audio_inputs)}'
# # filter


# maps = " ".join([f'-map {i}:a' for i in range(len(audio_inputs))])
# maps

# ! ffmpeg -i $v1 -i $v2 -filter_complex hstack=inputs=2 test9.webm

'ffmpeg -isync 6 -i testVideos/constructive_disagreement/r20230929_1040_student_012/1695991400747-1ee55745-2137-437a-b901-b237d0205f90-cam-audio-1695991408042 -isync 6 -i testVideos/constructive_disagreement/r20230929_1040_student_012/1695991400747-56cb9c8e-f466-41d0-82a6-1955bea4dfaa-cam-audio-1695991402070 -isync 6 -i testVideos/constructive_disagreement/r20230929_1040_student_012/1695991400747-5ede5172-0e84-428b-8e26-77ef8ecf7988-cam-audio-1695991520098 -isync 6 -i testVideos/constructive_disagreement/r20230929_1040_student_012/1695991400747-848f98c5-7eea-4b47-82a9-63efbb49194b-cam-audio-1695991402060 -isync 6 -i testVideos/constructive_disagreement/r20230929_1040_student_012/1695991400747-ac94f535-ef7b-4786-be39-f1df1a5b6995-cam-audio-1695991402053 -filter_complex "[0:a][1:a][2:a][3:a][4:a]amerge=inputs=5,pan=mono|c0<c0+c1+c2+c3+c4+c5+c6+c7+c8+c9|[a]" -map "[a]" test16.webm'

In [73]:
! $cmd

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

In [47]:
! ffprobe $a1

ffprobe version 6.0 Copyright (c) 2007-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --d